In [35]:
import api_keys
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [95]:
sp = spotipy.Spotify(
    client_credentials_manager = SpotifyClientCredentials(
        client_id=api_keys.CLIENT_ID,
        client_secret=api_keys.CLIENT_SECRET
    )    
)

def get_playlist_ids(date,offset):
    '''
    Get playlist ids for a particular day
    Inputs: date in ISO-8601 format
            offfset - page number
    Output: list of playlist ids
    '''
    return [d['id'] for d in sp.featured_playlists(
        locale=None,
        country='US',
        timestamp=date,
        limit=50,
        offset=offset*50
    )['playlists']['items']]
    
def get_playlist_tracks(id_):
    '''
    Get the tracks for a spotify playlist.
    Input: playlist ID
    Output: list of song IDs
    '''
    return [i['track']['id'] for i in sp.user_playlist('spotify',id_)['tracks']['items']]

def get_song_features(ids):
    '''
    Get song features for each song in a list
    '''
    return [sp.audio_features(_) for _ in ids]

In [84]:
get_playlist_ids('2019-08-06T19:47:31',0)

['37i9dQZF1DWWBHeXOYZf74',
 '37i9dQZF1DX0SM0LYsmbMT',
 '37i9dQZF1DX2A29LI7xHn1',
 '37i9dQZF1DWTfrr8pte1rT',
 '37i9dQZF1DX4y8h9WqDPAE',
 '37i9dQZF1DWXMg4uP5o3dm',
 '37i9dQZF1DWUxHPh2rEiHr',
 '37i9dQZF1DXadasIcsfbqh',
 '37i9dQZF1DWSfMe9z89s9B',
 '37i9dQZF1DX55dNU0PWnO5']

In [82]:
get_playlist_tracks('37i9dQZF1DWWBHeXOYZf74')[0:10]

['4PvbbMYL4fkToni5BLaYRb',
 '5fiR9Dy9hNXEPZOLo1kyNb',
 '1GZH9Sv6zCIse2GKihRHKy',
 '3vEOALYBUNQUMXpqhWx3hG',
 '6wE9qLKJu7DovH54rvrSVs',
 '3hMvsYZjU1pFH0xoywKXA9',
 '2LmQRj2QIzoyv9GcwGZslt',
 '3FsL491yGtc7Lwq1onHAsn',
 '5XR4V6uo2NCLkke1DyOcPY',
 '2urQEPkvz0ISpmqrKoSvwA']

In [87]:
sp.audio_features('5fiR9Dy9hNXEPZOLo1kyNb')

[{'danceability': 0.73,
  'energy': 0.634,
  'key': 2,
  'loudness': -5.116,
  'mode': 1,
  'speechiness': 0.297,
  'acousticness': 0.324,
  'instrumentalness': 6.16e-05,
  'liveness': 0.0712,
  'valence': 0.47,
  'tempo': 79.554,
  'type': 'audio_features',
  'id': '5fiR9Dy9hNXEPZOLo1kyNb',
  'uri': 'spotify:track:5fiR9Dy9hNXEPZOLo1kyNb',
  'track_href': 'https://api.spotify.com/v1/tracks/5fiR9Dy9hNXEPZOLo1kyNb',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5fiR9Dy9hNXEPZOLo1kyNb',
  'duration_ms': 219720,
  'time_signature': 4}]